# Deep learning, satellite image classification 

GeoAI, or geospatial artificial intelligence (AI), has become a trending topic and the frontier for
spatial analytics in Geography [(Li and Hsu, 2022)](https://www.mdpi.com/2220-9964/11/7/385/pdf). Although the field of AI has experienced highs and lows in the past decades, it has recently gained tremendous momentum because of breakthrough developments in deep (machine) learning, immense available computing power, and the pressing needs for mining and understanding big data. 


# Objectives 

The objective of the second *Case Study* is to show case how we can use GPU for satellite image classification. We will be discussing two case studies - (1) training a CNN model from scratch using Pytorch to detect landuse classfication from satellite images (2) using a pretrained computer vision model to underestand the "scenicness" of images. While using a GPU is a commonly integreated into deep learning libraries, we will also provide best practices to maximise your training efficiency. 

# Case Study 2.1. Classifying EuraSat images using Convolutional Neural Networks (CNNs)



## Convolutional Neural Networks (CNNs)

Convolutional Neural Networks (CNNs) are a type of artificial neural network that are designed to work with grid-structured data, such as an image, a speech signal, or a video. They are particularly effective for image and video classification, object detection and recognition, and natural language processing tasks.

The key components of a CNN are convolutional layers, activation functions, pooling layers, and fully connected layers. 

1. Convolutional layers: Convolutional layers are the building blocks of a CNN. They perform a convolution operation on the input data, where a small matrix (known as a filter or kernel) is moved across the input data, element-wise multiplication is performed between the elements of the filter and the input data, and then the results are summed up to produce a single output value. This process is repeated for every possible position of the filter, resulting in a set of outputs, called feature maps. Convolutional layers can extract features from the input data, such as edges, shapes, textures, etc.

2. Activation functions: Activation functions are used to introduce non-linearity into the network. They are applied element-wise to the output of the convolutional layer. The most commonly used activation functions in CNNs are Rectified Linear Unit (ReLU) and sigmoid.

3. Pooling layers: Pooling layers are used to reduce the spatial size of the feature maps, making the network less computationally expensive and more robust to changes in the position of objects in the input data. There are several types of pooling, including max pooling and average pooling. In max pooling, the maximum value in a region of the feature map is taken as the output, while in average pooling, the average value in a region is taken as the output.

4. Fully connected layers: The fully connected layers are used to make the final prediction using the features extracted by the convolutional and pooling layers. They perform a weighted sum of the inputs, followed by a non-linear activation function, and then produce the final output of the network.


The architecture of a CNN can be designed for a specific task by choosing the number of convolutional and fully connected layers, the size of the filters, the type of activation functions, and the type of pooling. The weights of the filters and the biases of the fully connected layers are learned from the training data using an optimization algorithm, such as stochastic gradient descent or Adam.



In [ ]:
# Importing libraries 


# python standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn standard functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


# standard imports for pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

# torchvision imports
import torchvision
import torchvision.transforms as transforms
import seaborn as sns 

import time

# Step 1: Data preparation and pre-processing 

- Data preperation. Download the EuroSat dataset from Pytorch. EuroSAT dataset is based on Sentinel-2 satellite images covering 13 spectral bands and consisting of 10 classes with 27000 labeled and geo-referenced samples. The classes are 'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River' and 'SeaLake'. 
- Data transformation - To prepare the data for our model, we need to convert image into the data structures that are recognisable on GPUs and Pytorch. We can also do data augumation to increase sample size, but we will not go into this in depth for this tutorial 
- Defining batch size - We can not pass the whole dataset into our model to train it, because our memory size is fixed and there is a high chance that our training data exceed the memory capacity of CPU or GPU, so we split the dataset into batches and instead of training the model on whole in a single phase. The batch size can be decided according to memory capacity, generally, it takes in power of 2. For example, the batch size can be 16, 32, 64, 128, 256, etc.


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#Define data pre-processing steps
transform = transforms.Compose(
    [
    #Resize images for (64*64)
    transforms.Resize((64,64)),
    #Converts images into Pytorch tensor 
    #Pytorch tensors are multi-dimensional arrays that can be processed on GPUs
    transforms.ToTensor(), 
    #Normalise the input data 
    #input data is transformed by subtracting the mean and dividing by the standard deviation for each channel. 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#Batch size defines the number of samples processed before the model is updated.
batch_size = 40 

#Loading EuraSAT and transform using the defined function 
dataset = torchvision.datasets.EuroSAT(root='./data', 
                                        download=True, transform=transform)

#Data loader creates a PyTorch data loader for a given dataset. 
#The data loader provides an efficient way to iterate over the data in the dataset
#and apply batch processing during training.      
#num_workers: defines the number of threads to use for loading the data. 
#If shuffle=True, the data loader will randomly shuffle the data before each epoch to ensure that the model sees a different set of samples each time it is trained.
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

#Classes -> we have 10 labels 
#'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River' 'SeaLake'
classes = data_loader.dataset.classes

split=len(dataset.targets)/4
train_len=int(len(dataset.targets)-split)
val_len=int(split) 

#Spliting dataset in 75% training, 25% for testing  
trainset,testset = torch.utils.data.random_split(dataset, [train_len,val_len])

#Create dataloader for training and testing dataset 
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)



# Step 2, Exploring images 
Our dataset consists of images in form of Tensors, imshow() method of matplotlib python library can be used to visualize images.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s},' for j in range(batch_size)))

# Step 3: Create your own Convnet Model for training

Let's create a simple CNN model with two convolution layers using Pytorch. 



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#Custom class extends the functionality of nn.Module class from PyTorch, 
#which provides the basic building blocks for creating neural networks in PyTorch. 
class Net(nn.Module):
    #Setting up layers in CNN 
    def __init__(self):
        #Calling function from nn.Module
        super().__init__()
        #A 2D convolutional layer with 3 input channels, 6 output, and kernel (filter size) size of 5x5 
        self.conv1 = nn.Conv2d(3, 6, 5)
        #A max-pooling layer with kernel size 2x2 and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        #Another convolution layer with 6 input channels, 16 output channels, and a kernel size of 5x5 
        self.conv2 = nn.Conv2d(6, 16, 5)
        #Three fully-connected linear layers for processing the output of the second convolution network 
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    #Define the foward pass of the network i.e. the computation performed on each input tensor. 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
from torchsummary import summary
summary(Net(), (3,64,64),device='cpu')

# Step 4: Inspecting CPU/GPU usage with PyTorch Profiler and TensorBoard



In [ ]:
# Define training function 
def train(model,data,criterion, optimizer,device ):
    # Copy the data to the device the model is on 
    inputs, labels = data[0].to(device=device), data[1].to(device=device)

    #Predict the output for given input
    outputs = model(inputs)

    #Compute the loss
    loss = criterion(outputs, labels)

    #Clear the previous gradients, compute gradients of all variables wrt loss
    optimizer.zero_grad()

    #Backpropagation, update weights
    loss.backward()

    #Update the parameters
    optimizer.step()


In [ ]:
#GPU ----------------------------
#Initialise model 
#Define device on cuda:0 
device = torch.device('cuda:0') 
model = Net().to(device=device)
#Define loss function 
loss_fn =  nn.CrossEntropyLoss().cuda()#Loss function computes the value between the predicted values and the labels. In this case, we are using Cross-Entropy loss, but many other loss functions are also avaible from nn. Such as focal loss 

#Define optimizer function 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Optimizer function aims to reduce the loss function's value by changing the weight vector values through backpropagation in neural networks. We are using Stochastic gradient decent as our optimiser, with learning rate 0.01 and momentum 0.9 

#Set random seed for reproducibility
torch.cuda.manual_seed(42)

#Profiler
with torch.profiler.profile(
       schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=3,
        repeat=4), 
        #Saving the profiling logs to a file that can be used by TensorBoard 
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/gpu_profile'),
        profile_memory=True,
        ) as prof:
    for step, batch_data in enumerate(train_loader,0):
        if step >= (2 + 2 + 3) * 4:
            break
        train(model =model , data =batch_data, criterion = loss_fn, optimizer = optimizer,device=device)
        prof.step()

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10)) 

In [ ]:
#CPU ----------------------------
#Reinitialise model, loss function, optimizer and random seed 
device = torch.device('cpu')
model = Net().to(device=device)
loss_fn =  nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
torch.manual_seed(42)

with torch.profiler.profile(
       schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=3,
        repeat=4),
        #Saving the profiling logs to a file that can be used by TensorBoard 
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/cpu_profile'),
        profile_memory=True,
        ) as prof:
    for step, batch_data in enumerate(train_loader,0):
        if step >= (2 + 2 + 3) * 4:
            break
        train(model =model , data =batch_data, criterion = loss_fn, optimizer = optimizer,device=device)
        prof.step()

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10)) 


In [ ]:
%load_ext tensorboard

# Comparing model performance with a fine-tune model



In [ ]:
vgg11_bn = torchvision.models.vgg11_bn(weights=True)
# Freeze weights of all layers except the new classification layer
for param in vgg11_bn.parameters():
    param.requires_grad = False 
num_ftrs = vgg11_bn.classifier[6].in_features
# Replace the final classfication layer 
vgg11_bn.classifier[6] = nn.Linear(num_ftrs,len(classes))
vgg11_bn.classifier[6].requires_grad = True  

In [ ]:
#VGG16:GPU --------

#GPU ----------------------------
#Initialise model 
#Define device on cuda:0 
device = torch.device('cuda:0') 
#Change model to vgg11_bn
model = vgg11_bn.to(device=device)
#Define loss function 
loss_fn =  nn.CrossEntropyLoss().cuda()#Loss function computes the value between the predicted values and the labels. In this case, we are using Cross-Entropy loss, but many other loss functions are also avaible from nn. Such as focal loss 

#Define optimizer function 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Optimizer function aims to reduce the loss function's value by changing the weight vector values through backpropagation in neural networks. We are using Stochastic gradient decent as our optimiser, with learning rate 0.01 and momentum 0.9 

#Set random seed for reproducibility
torch.cuda.manual_seed(42)

#Profiler
with torch.profiler.profile(
       schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=3,
        repeat=4), 
        #Saving the profiling logs to a file that can be used by TensorBoard 
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/gpu_vgg'),
        profile_memory=True,
        ) as prof:
    for step, batch_data in enumerate(train_loader,0):
        if step >= (2 + 2 + 3) * 4:
            break
        train(model =model , data =batch_data, criterion = loss_fn, optimizer = optimizer,device=device)
        prof.step()

In [ ]:
#VGG16:CPU --------

#CPU ----------------------------
#Initialise model 
#Define device on cuda:0 
device = torch.device('cpu')
#Change model to vgg11_bn
model = vgg11_bn.to(device=device)
#Define loss function 
loss_fn =  nn.CrossEntropyLoss().cuda()#Loss function computes the value between the predicted values and the labels. In this case, we are using Cross-Entropy loss, but many other loss functions are also avaible from nn. Such as focal loss 

#Define optimizer function 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Optimizer function aims to reduce the loss function's value by changing the weight vector values through backpropagation in neural networks. We are using Stochastic gradient decent as our optimiser, with learning rate 0.01 and momentum 0.9 

#Set random seed for reproducibility
torch.cuda.manual_seed(42)

#Profiler
with torch.profiler.profile(
       schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=3,
        repeat=4), 
        #Saving the profiling logs to a file that can be used by TensorBoard 
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/cpu_vgg'),
        profile_memory=True,
        ) as prof:
    for step, batch_data in enumerate(train_loader,0):
        if step >= (2 + 2 + 3) * 4:
            break
        train(model =model , data =batch_data, criterion = loss_fn, optimizer = optimizer,device=device)
        prof.step()

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, device=device):
    
    # Initialize time 
    since = time.time()

    # Initialize reporting metrics 
    train_acc_history,val_acc_history,train_loss_history,val_loss_history = [],[],[],[]
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
        
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).type(torch.float).item()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
                train_acc_history.append(epoch_acc)     
                train_loss_history.append(epoch_loss)
            else: 
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))


    return train_acc_history,val_acc_history,train_loss_history,val_loss_history,time_elapsed

In [ ]:
# Dictionary of dataloader 
dataloader_all = {}
dataloader_all['train'] = train_loader
dataloader_all['val'] = test_loader


In [ ]:
#VGG16
#CPU----------------------------------------------------------------------------------------------------------------
device = torch.device('cpu') 
vgg11_cpu = vgg11_bn.to(device=device)
loss_fn =  nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg11_cpu.parameters(), lr=0.01, momentum=0.9) 
train_acc_history,val_acc_history,train_loss_history,val_loss_history,vgg_cpu_time  = train_model(vgg11_cpu,dataloader_all,loss_fn, optimizer, num_epochs=10, device=device) 
#GPU----------------------------------------------------------------------------------------------------------------
device = torch.device('cuda:0') 
vgg11_gpu = vgg11_bn.to(device=device)
loss_fn =  nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(vgg11_gpu.parameters(), lr=0.01, momentum=0.9) 
train_acc_history,val_acc_history,train_loss_history,val_loss_history,vgg_gpu_time  = train_model(vgg11_gpu,dataloader_all,loss_fn, optimizer, num_epochs=10, device=device)


In [ ]:
#Baseline
#CPU----------------------------------------------------------------------------------------------------------------
device = torch.device('cpu')
base_cpu = Net().to(device=device)
loss_fn =  nn.CrossEntropyLoss()
optimizer = optim.SGD(base_cpu.parameters(), lr=0.01, momentum=0.9)
base_train_acc_history,base_val_acc_history,base_train_loss_history,base_val_loss_history,baseline_cpu_time  = train_model(base_cpu,dataloader_all,loss_fn, optimizer, num_epochs=10, device=device)
#GPU----------------------------------------------------------------------------------------------------------------
device = torch.device('cuda:0')
base_gpu = Net().to(device=device)
loss_fn =  nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(base_gpu.parameters(), lr=0.01, momentum=0.9)
base_train_acc_history,base_val_acc_history,base_train_loss_history,base_val_loss_history,baseline_gpu_time  = train_model(base_gpu,dataloader_all,loss_fn, optimizer, num_epochs=10, device=device)

In [ ]:
#Plot compute time 
compute_time = pd.DataFrame([baseline_cpu_time,baseline_gpu_time,vgg_cpu_time,vgg_gpu_time],columns=['Time'])
compute_time['Model'] = ['baseline','baseline','VGG16','VGG16']
compute_time['Mode'] = ['CPU','GPU','CPU','GPU']

sns.set_theme(style="whitegrid")
g= sns.catplot(data=compute_time, kind='bar',x='Model',y='Time',hue='Mode')
g.set_axis_labels("", "Total Time (Seconds)")


# Visualise results  

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(20,20))
def show_heatmap(test_loader,model,ax,name):
    heatmap = pd.DataFrame(data=0,index=classes,columns=classes)
    with torch.no_grad():
        number_corrects = 0
        number_samples = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            number_corrects += (predicted==labels).sum().item()
            number_samples += labels.size(0)
            for i in range(len(labels)):
                true_label = labels[i].item()
                predicted_label = predicted[i].item()
                heatmap.iloc[true_label,predicted_label] += 1
    sns.heatmap(heatmap, annot=True, fmt="d",cmap="YlGnBu",ax=ax) 
    ax.set_title(f'{name}, Overall accuracy {(number_corrects / number_samples)*100}%') 

show_heatmap(test_loader,vgg11_gpu,ax[0],"VGG16")
show_heatmap(test_loader,base_gpu,ax[1],"Baseline")

In [ ]:
#Plotting Loss  
fig,ax = plt.subplots(1,2,figsize=(10,5))
vgg_loss_array = zip(train_loss_history,val_loss_history)
vgg_loss_df=pd.DataFrame(vgg_loss_array,columns=['train','test'])
vgg_loss_df.plot(ax=ax[0])
ax[0].set_title('vgg16')
ax[0].set_ylim(0,2)

baseline_loss_array = zip(base_train_loss_history,base_val_loss_history)
baseline_loss_df = pd.DataFrame(baseline_loss_array,columns=['train','test'])
baseline_loss_df.plot(ax=ax[1])
ax[1].set_title('Baseline')
ax[1].set_ylim(0,2)

In [ ]:
#Plotting Accuracy 
fig,ax = plt.subplots(1,2,figsize=(10,5))
vgg_acc_array = zip(train_acc_history,val_acc_history)
vgg_acc_df=pd.DataFrame(vgg_acc_array,columns=['train','test'])
vgg_acc_df.plot(ax=ax[0])
ax[0].set_title('vgg16')
ax[0].set_ylim(0,1)

baseline_acc_array = zip(base_train_acc_history,base_val_acc_history)
baseline_acc_df = pd.DataFrame(baseline_acc_array,columns=['train','test'])
baseline_acc_df.plot(ax=ax[1])
ax[1].set_title('Baseline')
ax[1].set_ylim(0,1)
